# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of ABBA with python.

If you have executed the previous notebook, you can re-open the saved state file.

This notebook shows how to convert the pixel position of a slice to its coordinates in the atlas (forth and back).


In [1]:
# core dependencies
import os
import time
from pathlib import Path


from bg_atlasapi import show_atlases
from bg_atlasapi import utils

from abba_python.Abba import Abba

In [2]:
# -- NOT HEADLESS
abba = Abba('Adult Mouse Brain - Allen Brain Atlas V3p1') # You may have to click a window!
abba.show_bdv_ui()  # creates and show a bdv view

save_dir = os.path.join(os.getcwd(), 'temp', 'notebook0', 'state') # execute the first notebook to get this file!
abba.state_load(save_dir+"/state.abba") # full absolute path needed

# ALL COMMANDS ARE PERFORMED ON SELECTED SLICES!!
# since we want to register all of them, we select all of them
abba.select_all_slices()

# change display settings : channel index, min, max
abba.change_display_settings(0, 0, 500)
abba.change_display_settings(1, 0, 1200)

# YOU MAY NEED TO CLICK ON THE UI TO FINISH THE EXECUTION OF THIS CELL (angle change notification)

[java.lang.Enum.toString] 15:57:50.128 [SciJava-75308740-Thread-0] DEBUG sc.fiji.bdvpg.scijava.services.SourceAndConverterService - Plugin found for entity class Displaysettings
[java.lang.Enum.toString] 15:57:50.509 [SciJava-75308740-Thread-0] INFO sc.fiji.bdvpg.scijava.services.SourceAndConverterService - Spimdata mpicbg.spim.data.SpimData@1720c9f3 is using the Bdv Playground global cache.
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: ba [ch.epfl.biop.atlas.mouse.allen.ccfv3p1.command.AllenBrainAdultMouseAtlasCCF2017v3p1Command]
[java.lang.Enum.toString] 15:57:50.775 [SciJava-75308740-Thread-0] DEBUG sc.fiji.bdvpg.scijava.services.SourceAndConverterService - Source already registered
[java.lang.Enum.toString] 15:57:50.775 [SciJava-75308740-Thread-0] DEBUG sc.fiji.bdvpg.scijava.services.SourceAndConverterService - Source already registered
[java.lang.Enum.toString] 15:57:50.783 [SciJava-75308740-Thread-0] DEBUG sc.fiji.bdvpg.scijava.services.SourceAndConverterServic

[java.lang.Enum.toString] Keymap list file C:\Users\chiarutt\AppData\Roaming\fiji\bigdataviewer\config\keymaps\keymaps.yaml not found. Using builtin styles.[java.lang.Enum.toString] 
[java.lang.Enum.toString] Appearance settings file C:\Users\chiarutt\AppData\Roaming\fiji\bigdataviewer\config/appearance.yaml not found. Using defaults.[java.lang.Enum.toString] 
[java.lang.Enum.toString] [ERROR] Exception during event handling:
	[Event] org.scijava.display.event.DisplayActivatedEvent
	context = org.scijava.Context@75308740
	consumed = false
	display = plugin:org.scijava.table.DefaultTableDisplay: type=interface org.scijava.table.Table, name=ABBA - Load State, objects={[[success], [true]]}
	[Subscriber] Foreground
	[Method] protected void net.imagej.plugins.tools.AbstractColorTool.onEvent(org.scijava.display.event.DisplayActivatedEvent)
java.lang.AbstractMethodError: Method net/imagej/legacy/display/LegacyImageDisplayService.getActiveDatasetView()Lnet/imagej/display/DatasetView; is abstra

In [3]:
# programmatic way to show (or hide) sections and channels
abba.get_bdv_view().setSelectedSlicesVisibility(True)
abba.get_bdv_view().setSelectedSlicesVisibility(0, True)

In [4]:
mp = abba.mp
mp.selectSlice(mp.getSlices()) # select all slices

In [5]:
mp.deselectSlice(mp.getSlices()) # deselect all

In [6]:
mp.selectSlice(mp.getSlices().get(2)) # select the third slice

In [7]:
# The slices are always sorted from small z to high z. To keep track of who's who, reference them before moving them
slice_0 = mp.getSlices().get(0) 
slice_1 = mp.getSlices().get(1)
slice_2 = mp.getSlices().get(2)

In [8]:

from jpype.types import JString, JArray, JDouble

# Get transformation
transform_pix_to_atlas = slice_0.getSlicePixToCCFRealTransform()

DoubleArray = JArray(JDouble)

coordInImage = DoubleArray(3)
coordInCCF = DoubleArray(3)

coordInImage[0] = 500 # X (pixel)
coordInImage[1] = 500 # Y
coordInImage[2] = 0 # Z

transform_pix_to_atlas.inverse().apply(coordInImage,coordInCCF)

print('CCF coord (mm):'+str(coordInCCF))


CCF coord (mm):[3.187182987823669, 7.3321108933805785, 4.386380823174239]


In [9]:
# We can also find the coordinates of the atlas into a slice
# a coordinate in z far from 0, means that it's not in the slice, but far out
# See https://forum.image.sc/t/coordinates-in-slice-to-atlas-using-abba-python/79513 

coordInCCF = DoubleArray(3)
coordInCCF[0] = 6.60 # X (pixel)
coordInCCF[1] = 4.31 # Y
coordInCCF[2] = 5.58 # Z

coordInImage = DoubleArray(3)

for idx in range(0,mp.getSlices().size()):
    # Get transformation
    transform_pix_to_atlas = mp.getSlices().get(idx).getSlicePixToCCFRealTransform()
    transform_pix_to_atlas.apply(coordInCCF, coordInImage)
    print('Slice['+str(idx)+'] coord (pixel):'+str(coordInImage))
    


Slice[0] coord (pixel):[360.66306173890723, 242.412401575719, 4.072907353315796]
Slice[1] coord (pixel):[357.35359954895216, 286.9910866959781, 3.292809872735901]
Slice[2] coord (pixel):[393.83202866254845, 328.18900628142904, 4.103866430977466]
Slice[3] coord (pixel):[429.01572962981567, 302.50628733620937, 3.932179606174112]
Slice[4] coord (pixel):[446.18591265307435, 308.1064306291544, 1.973810749134363]
Slice[5] coord (pixel):[464.4364692957545, 326.94001551905103, 0.9712097461078688]
Slice[6] coord (pixel):[478.91913492067295, 345.5789057544712, -0.005233937897105786]
Slice[7] coord (pixel):[465.73330309209643, 312.45662603493025, -1.0583766845158031]
Slice[8] coord (pixel):[494.5616390029002, 354.52886542017274, -2.164965177778075]
Slice[9] coord (pixel):[485.11188876789987, 344.7103654364204, -2.4685074486167267]
Slice[10] coord (pixel):[502.3028782042699, 351.4634885691764, -3.25884377054199]
Slice[11] coord (pixel):[485.43834591756405, 361.1147144605723, -4.278646241996325]
Sl

In [10]:
# The slice indexed 6 should be the one which is the nearest to the atlas point.
# Note that the values in Z are small because, in absence of 3D image, 
# each section is assumed to have a size of 1 mm

In [11]:
# You can normalize the slices sections by expanding them to cover the space between neighboring slices, no more, no less
abba.select_all_slices()
abba.set_slices_thickness_match_neighbors() # not critical, but for 3d reconstruction it will allow for each slice to occupy the place available between its neighbors
# The coordinates will be different in Z
coordInImage = DoubleArray(3)

for idx in range(0,mp.getSlices().size()):
    # Get transformation
    transform_pix_to_atlas = mp.getSlices().get(idx).getSlicePixToCCFRealTransform()
    transform_pix_to_atlas.apply(coordInCCF, coordInImage)
    print('Slice['+str(idx)+'] coord (pixel):'+str(coordInImage))

Slice[0] coord (pixel):[360.66306173890723, 242.412401575719, 4.072907353315796]
Slice[1] coord (pixel):[357.35359954895216, 286.9910866959781, 3.292809872735901]
Slice[2] coord (pixel):[393.83202866254845, 328.18900628142904, 4.103866430977466]
Slice[3] coord (pixel):[429.01572962981567, 302.50628733620937, 3.932179606174112]
Slice[4] coord (pixel):[446.18591265307435, 308.1064306291544, 1.973810749134363]
Slice[5] coord (pixel):[464.4364692957545, 326.94001551905103, 0.9712097461078688]
Slice[6] coord (pixel):[478.91913492067295, 345.5789057544712, -0.005233937897105786]
Slice[7] coord (pixel):[465.73330309209643, 312.45662603493025, -1.0583766845158031]
Slice[8] coord (pixel):[494.5616390029002, 354.52886542017274, -2.164965177778075]
Slice[9] coord (pixel):[485.11188876789987, 344.7103654364204, -2.4685074486167267]
Slice[10] coord (pixel):[502.3028782042699, 351.4634885691764, -3.25884377054199]
Slice[11] coord (pixel):[485.43834591756405, 361.1147144605723, -4.278646241996325]
Sl